In [1]:
import pandas as pd
import numpy as np

In [11]:
#COMANDO PARA NÃO QUEBRAR LINHA NO DATAFRAME
pd.set_option('expand_frame_repr', False)

In [2]:
#LOAD:
df2 = pd.read_csv('dados/IFDATA_PF_db.csv', sep=';')

In [3]:
#QUESTOES
#Q1 - MAIORES INSTITUIÇÕES - PF
#Q2 - PRINCIPAIS PRODUTOS / INAD - PF
#Q3 - RELEVÂNCIA OPERAÇÃO CARTÃO DE CRÉDITO - PF

In [12]:
Q1 = df2.assign(
    vencido = np.where(df2['nomecoluna'].str.contains('VENCIDO'),df2['saldo'],0),
    vencer = np.where(df2['nomecoluna'].str.contains('VENCER'),df2['saldo'],0),
    total = df2['saldo']
    ).groupby('codinst').agg({'vencido':sum, 'vencer':sum, 'total':sum}).sort_values(by='total', ascending=False)

inad = Q1['vencido'] / Q1['total']  
Q1 = Q1.assign(inad=inad)

# TRATANDO VALORES NÃO NUMÉRICOS PARA NaN
Q1['inad'] = pd.to_numeric(Q1['inad'], errors='coerce')

# PREENCHENDO NaN COM 0 E FORMATANDO ESTES E DEMAIS COM PERCENTUAL
Q1['inad'] = Q1['inad'].fillna(0).apply(lambda x: '{:.2%}'.format(x))

# TRATANDO VALORES NUMÉRICOS PARA BILHÕES OU MILHÕES
def convert_to_billion(x):
    if x > 1000000000:
        return str(round((x/1000000000),2)) + 'B'
    else: 
        return str(round((x/1000000),2)) + 'M'

Q1['vencido'] = Q1['vencido'].apply(lambda x : convert_to_billion(x))
Q1['vencer'] = Q1['vencer'].apply(lambda x : convert_to_billion(x))
Q1['total'] = Q1['total'].apply(lambda x : convert_to_billion(x))

print(Q1)

         vencido   vencer    total   inad
codinst                                  
C0051626   5.72B  788.44B  794.16B  0.72%
C0049906  10.16B   542.5B  552.66B  1.84%
C0010069  17.72B  387.59B  405.31B  4.37%
C0010045   12.8B  339.05B  351.86B  3.64%
C0030379   9.03B  270.82B  279.86B  3.23%
...          ...      ...      ...    ...
41592532    0.0M     0.0M     0.0M  0.00%
40819834    0.0M     0.0M     0.0M  0.00%
40768766    0.0M     0.0M     0.0M  0.00%
40434681    0.0M     0.0M     0.0M  0.00%
48176336    0.0M     0.0M     0.0M  0.00%

[1358 rows x 4 columns]


In [13]:
Q2 = df2.assign(
    vencido = np.where(df2['nomecoluna'].str.contains('VENCIDO'),df2['saldo'],0),
    vencer = np.where(df2['nomecoluna'].str.contains('VENCER'),df2['saldo'],0),
    total = df2['saldo']
    ).groupby('grupo').agg({'vencido':sum, 'vencer':sum, 'total':sum}).sort_values(by='total', ascending=False)

inad = Q2['vencido'] / Q2['total']  
Q2 = Q2.assign(inad=inad)

# TRATANDO VALORES NÃO NUMÉRICOS PARA NaN
Q2['inad'] = pd.to_numeric(Q2['inad'], errors='coerce')

# PREENCHENDO NaN COM 0 E FORMATANDO ESTES E DEMAIS COM PERCENTUAL
Q2['inad'] = Q2['inad'].fillna(0).apply(lambda x: '{:.2%}'.format(x))

# TRATANDO VALORES NUMÉRICOS PARA BILHÕES OU MILHÕES
Q2['vencido'] = Q2['vencido'].apply(lambda x : convert_to_billion(x))
Q2['vencer'] = Q2['vencer'].apply(lambda x : convert_to_billion(x))
Q2['total'] = Q2['total'].apply(lambda x : convert_to_billion(x))

print(Q2)

                                    vencido   vencer    total    inad
grupo                                                                
Habitação                             1.51B  933.13B  934.64B   0.16%
Empréstimo com Consignação em Folha   3.62B  586.68B  590.29B   0.61%
Rural e Agroindustrial                2.22B  489.79B  492.01B   0.45%
Cartão de Crédito                    52.45B  439.15B   491.6B  10.67%
Empréstimo sem Consignação em Folha  10.64B  254.33B  264.97B   4.02%
Veículos                              4.75B  255.37B  260.11B   1.82%
Outros Créditos                      13.09B  166.76B  179.85B   7.28%


In [43]:
Q3 = df2.assign(
    total = df2['saldo']
    ).groupby('grupo', as_index=False).agg({'total':sum}).sort_values(by='total', ascending=False)
total = Q3['total'].sum()

# CALCULANDO A REPRESENTATIVIDADE SOBRE O TOTAL E FORMATANDO O PERCENTUAL
Q3['% do total'] = Q3['total'].apply(lambda x: (x/total))
Q3['% do total'] = Q3['% do total'].apply(lambda x: '{:.2%}'.format(x))

# TRATANDO VALORES NUMÉRICOS PARA BILHÕES OU MILHÕES
Q3['total'] = Q3['total'].apply(lambda x : convert_to_billion(x))

print(Q3)


                                 grupo    total % do total
3                            Habitação  934.64B     29.08%
1  Empréstimo com Consignação em Folha  590.29B     18.37%
5               Rural e Agroindustrial  492.01B     15.31%
0                    Cartão de Crédito   491.6B     15.30%
2  Empréstimo sem Consignação em Folha  264.97B      8.25%
6                             Veículos  260.11B      8.09%
4                      Outros Créditos  179.85B      5.60%


In [41]:
#QUESTOES - PF
#Q1 - MAIORES INSTITUIÇÕES - PF
#Q2 - PRINCIPAIS PRODUTOS / INAD - PF
#Q3 - RELEVÂNCIA OPERAÇÃO CARTÃO DE CRÉDITO - PF
print('As 5 maiores instituições, em tamanho da carteira de crédito para pessoa física, são: \n {} \n '.format(Q1.head(5)))
print('Os 5 principais produtos da carteira de crédito para pessoa física e suas respectivas taxas de inadimplência, são: \n {} \n '.format(Q2.head(5)))
print('A relevância de operações com Cartão de crédito, dentro da carteira de crédito para pessoa física, é de: \n {} \n '.format(Q3[Q3['grupo'].str.contains("Cartão de Crédito")]))

As 5 maiores instituições, em tamanho da carteira de crédito para pessoa física, são: 
          vencido   vencer    total   inad
codinst                                  
C0051626   5.72B  788.44B  794.16B  0.72%
C0049906  10.16B   542.5B  552.66B  1.84%
C0010069  17.72B  387.59B  405.31B  4.37%
C0010045   12.8B  339.05B  351.86B  3.64%
C0030379   9.03B  270.82B  279.86B  3.23% 
 
Os 5 principais produtos da carteira de crédito para pessoa física e suas respectivas taxas de inadimplência, são: 
                                     vencido   vencer    total    inad
grupo                                                                
Habitação                             1.51B  933.13B  934.64B   0.16%
Empréstimo com Consignação em Folha   3.62B  586.68B  590.29B   0.61%
Rural e Agroindustrial                2.22B  489.79B  492.01B   0.45%
Cartão de Crédito                    52.45B  439.15B   491.6B  10.67%
Empréstimo sem Consignação em Folha  10.64B  254.33B  264.97B   4.02% 
 
A rel